<a href="https://colab.research.google.com/github/veerendra12/CS598-DL4H-Project/blob/main/notebooks/NIHPreprocessor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import import_ipynb

import pandas as pd 
from sklearn.model_selection import train_test_split
import glob

from Configuration import CONFIG

importing Jupyter notebook from Configuration.ipynb


In [5]:
def transform_to_onehot(entry):
    labels = CONFIG['CLASS_LABELS']
    label_dict = {label:0 for label in labels}
    label_dict['Image Index'] = entry['Image Index']
    label_dict['Patient ID'] = entry['Patient ID']
    label_lsit = entry['Finding Labels'].split('|')
    for label in label_lsit:
        label_dict[label] = 1
        
    return pd.Series(label_dict)

def convert_NIH_labels_to_onehotvectors():
  BASE_DIR = CONFIG['BASE_DIR']
  NIH_DATASET_DRIVE_DIR = CONFIG['NIH_DATASET_DRIVE_DIR']
  label_file = NIH_DATASET_DRIVE_DIR + CONFIG['NIH_RAW_LABEL_DATA_FILE_NAME']

  !mkdir -p $BASE_DIR
  !cp -v $label_file $BASE_DIR

  nih_label_data_raw = pd.read_csv(BASE_DIR + CONFIG['NIH_RAW_LABEL_DATA_FILE_NAME'])
  print(nih_label_data_raw.head())

  nih_label_data_onehot = nih_label_data_raw.apply(transform_to_onehot, axis=1)
  
  nih_label_data_onehot.to_csv(CONFIG['NIH_CXR_DATA_ONEHOT_FULL_CSV_FILE_NAME'], index=False)
  return nih_label_data_onehot

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y]
0,00000001_000.png,Cardiomegaly,0,1,57,M,PA,2682,2749,0.143,0.143
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168
3,00000002_000.png,No Finding,0,2,80,M,PA,2500,2048,0.171,0.171
4,00000003_001.png,Hernia,0,3,74,F,PA,2500,2048,0.168,0.168
...,...,...,...,...,...,...,...,...,...,...,...
112115,00030801_001.png,Mass|Pneumonia,1,30801,38,M,PA,2048,2500,0.168,0.168
112116,00030802_000.png,No Finding,0,30802,28,M,PA,2048,2500,0.168,0.168
112117,00030803_000.png,No Finding,0,30803,42,F,PA,2048,2500,0.168,0.168
112118,00030804_000.png,No Finding,0,30804,29,F,PA,2048,2500,0.168,0.168


In [ ]:
def prepare_NIH_dataset(images_folder, name = 'NIH_CXR_Data_OneHot.csv', save = False):
    image_list = glob.glob(glob.escape(images_folder) + '/*')
    image_list = [image.split('/')[-1] for image in image_list]
    chest_xray_onehot_full_df = pd.read_csv(CONFIG['NIH_CXR_DATA_ONEHOT_FULL_CSV'])
    chest_xray_onehot_full_df = chest_xray_onehot_full_df.set_index('Image Index')
    chest_xray_onehot_subset_df = chest_xray_onehot_full_df.loc[image_list].reset_index()
    
    if save:
      chest_xray_onehot_subset_df.to_csv(CONFIG['BASE_DIR'] + name, index=False)

    return chest_xray_onehot_subset_df


In [ ]:
def generate_train_test_split(images_onehot_df, save = False):
    sample_size = CONFIG['SAMPLE_SIZE']
    print("Length of Original  Dataset : " , len(images_onehot_df))
    
    image_onehot_sampled_df = images_onehot_df.sample(frac=sample_size, 
                                                     replace=False, 
                                                     random_state=CONFIG['RANDOM_SEED'])
    print("Length of Sample  Dataset : " , len(image_onehot_sampled_df))
    
    train, validation = train_test_split(image_onehot_sampled_df, 
                                         test_size=CONFIG['TEST_TRAIN_SPLIT'], 
                                         random_state=CONFIG['RANDOM_SEED'], 
                                         shuffle=True)
    if save:
      train.to_csv(CONFIG['BASE_DIR'] + "train_" + str(sample_size) + ".csv", index=False)
      validation.to_csv(CONFIG['BASE_DIR'] + "validation_" + str(sample_size) + ".csv", index=False)
  
    return train, validation